# Trabajo Práctico N°3: NLP
## Embeddings con Gensim





In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Datos
El dataset será el libro Harry Potter y la Piedra Filosofal.

In [4]:
import re

# Leer el archivo como un solo string
with open('/content/Harry Potter 1.txt', 'r') as f:
    texto = f.read()

# Reemplazar "\n\n" por "." y "\n" por " "
texto = texto.replace('\n\n', '. ').replace('\n', ' ')

# Eliminar los guiones, signos de exclamación, interrogación
texto = re.sub(r'[-—¡¿!?]', '', texto)

# Si quieres escribir el texto modificado a un nuevo archivo:
with open('/content/Harry Potter 1_modificado.txt', 'w') as f:
    f.write(texto)




In [5]:
import pandas as pd
import nltk

# Descarga el tokenizador de NLTK si aún no lo has hecho
nltk.download('punkt')

# Cargar el tokenizador de oraciones español de NLTK
spanish_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

sentences = []

with open('/content/Harry Potter 1_modificado.txt', 'r') as f:
    for line in f:
        line_sentences = spanish_tokenizer.tokenize(line)
        sentences.extend(line_sentences)

# Convierte la lista de oraciones en un DataFrame
df = pd.DataFrame(sentences, columns=['sentence'])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [6]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Descarga el conjunto de stopwords de NLTK si aún no lo has hecho
nltk.download('stopwords')

# Descarga el tokenizador de NLTK si aún no lo has hecho
nltk.download('punkt')

# Cargar el tokenizador de oraciones español de NLTK
spanish_tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')

# Carga las stopwords en español de NLTK
stop_words = set(stopwords.words('spanish'))

sentences = []
sentences_no_stopwords = []

with open('/content/Harry Potter 1_modificado.txt', 'r') as f:
    for line in f:
        line_sentences = spanish_tokenizer.tokenize(line)
        sentences.extend(line_sentences)
        
        # Eliminar las stopwords de cada oración y agregarlas a sentences_no_stopwords
        for sentence in line_sentences:
            word_tokens = word_tokenize(sentence, language='spanish')
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            sentences_no_stopwords.append(' '.join(filtered_sentence))

# Convierte la lista de oraciones en un DataFrame
df = pd.DataFrame(sentences, columns=['sentence'])

# Convierte la lista de oraciones sin stopwords en otro DataFrame
df_no_stopwords = pd.DataFrame(sentences_no_stopwords, columns=['sentence'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
df.head()

,sentence
0,HARRY POTTER Y LA PIEDRA FILOSOFAL J.K.
1,ROWLING Harry Potter se ha quedado huérfano y ...
2,"Harry se siente muy triste y solo, hasta que u..."
3,En ella le comunican que ha sido aceptado como...
4,"A partir de ese momento, la suerte de Harry da..."


In [8]:
df.head()

,sentence
0,HARRY POTTER Y LA PIEDRA FILOSOFAL J.K.
1,ROWLING Harry Potter se ha quedado huérfano y ...
2,"Harry se siente muy triste y solo, hasta que u..."
3,En ella le comunican que ha sido aceptado como...
4,"A partir de ese momento, la suerte de Harry da..."


In [9]:
df_no_stopwords.head()

,sentence
0,HARRY POTTER Y LA PIEDRA FILOSOFAL J.K .
1,ROWLING Harry Potter quedado huérfano vive cas...
2,"Harry triste solo , buen día recibe carta camb..."
3,En comunican sido aceptado alumno colegio inte...
4,"A partir momento , suerte Harry da vuelco espe..."


In [10]:
print("Cantidad de documentos:", df.shape[0])
print("Cantidad de documentos en el df sin stopwords:", df_no_stopwords.shape[0])

Cantidad de documentos: 5859
Cantidad de documentos en el df sin stopwords: 5859


### 1 - Preprocesamiento

In [11]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
sentence_tokens_no_stopwords = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

for _, row in df_no_stopwords[:None].iterrows():
    sentence_tokens_no_stopwords.append(text_to_word_sequence(row[0]))

In [12]:
# Demos un vistazo
sentence_tokens[2:4]

[['harry',
  'se',
  'siente',
  'muy',
  'triste',
  'y',
  'solo',
  'hasta',
  'que',
  'un',
  'buen',
  'día',
  'recibe',
  'una',
  'carta',
  'que',
  'cambiará',
  'su',
  'vida',
  'para',
  'siempre'],
 ['en',
  'ella',
  'le',
  'comunican',
  'que',
  'ha',
  'sido',
  'aceptado',
  'como',
  'alumno',
  'en',
  'el',
  'colegio',
  'interno',
  'hogwarts',
  'de',
  'magia',
  'y',
  'hechicería']]

In [13]:
sentence_tokens_no_stopwords[2:4]

[['harry',
  'triste',
  'solo',
  'buen',
  'día',
  'recibe',
  'carta',
  'cambiará',
  'vida',
  'siempre'],
 ['en',
  'comunican',
  'sido',
  'aceptado',
  'alumno',
  'colegio',
  'interno',
  'hogwarts',
  'magia',
  'hechicería']]

### 2 - Crear los vectores (word2vec)

In [14]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [15]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores 
                     negative=10,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

w2v_model_no_stopwords = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores 
                     negative=10,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

w2v_model_cbow = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores 
                     negative=10,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

w2v_model_no_stopwords_cbow = Word2Vec(min_count=10,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=50,       # dimensionalidad de los vectores 
                     negative=10,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram



In [16]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)
w2v_model_no_stopwords.build_vocab(sentence_tokens_no_stopwords)
w2v_model_cbow.build_vocab(sentence_tokens)
w2v_model_no_stopwords_cbow.build_vocab(sentence_tokens_no_stopwords)

In [17]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)
print("Cantidad de docs en el corpus:", w2v_model_no_stopwords.corpus_count)

Cantidad de docs en el corpus: 5859
Cantidad de docs en el corpus: 5859


### 3 - Entrenar el modelo generador

In [18]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 408420.375
Loss after epoch 1: 322676.6875
Loss after epoch 2: 320022.1875
Loss after epoch 3: 320710.125
Loss after epoch 4: 315064.0
Loss after epoch 5: 304166.875
Loss after epoch 6: 273852.75
Loss after epoch 7: 258974.0
Loss after epoch 8: 256973.5
Loss after epoch 9: 254017.25
Loss after epoch 10: 252760.5
Loss after epoch 11: 249214.75
Loss after epoch 12: 249321.5
Loss after epoch 13: 246110.5
Loss after epoch 14: 236716.5
Loss after epoch 15: 215066.0
Loss after epoch 16: 214972.5
Loss after epoch 17: 215640.5
Loss after epoch 18: 214767.0
Loss after epoch 19: 214058.0
Loss after epoch 20: 214190.0
Loss after epoch 21: 213115.5
Loss after epoch 22: 212362.5
Loss after epoch 23: 212757.5
Loss after epoch 24: 212424.5
Loss after epoch 25: 211708.0
Loss after epoch 26: 212327.5
Loss after epoch 27: 211880.0
Loss after epoch 28: 210178.5
Loss after epoch 29: 210935.5
Loss after epoch 30: 210508.5
Loss after epoch 31: 211275.5
Loss after epoch 32: 209111.0
Loss 

(1982726, 3914200)

In [19]:
w2v_model_no_stopwords.train(sentence_tokens_no_stopwords,
                 total_examples=w2v_model_no_stopwords.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 304351.71875
Loss after epoch 1: 182541.90625
Loss after epoch 2: 181990.5625
Loss after epoch 3: 181610.4375
Loss after epoch 4: 180409.9375
Loss after epoch 5: 185072.8125
Loss after epoch 6: 183631.5
Loss after epoch 7: 179515.5
Loss after epoch 8: 173701.25
Loss after epoch 9: 169761.5
Loss after epoch 10: 164519.5
Loss after epoch 11: 143274.875
Loss after epoch 12: 140518.0
Loss after epoch 13: 137989.5
Loss after epoch 14: 136767.0
Loss after epoch 15: 134819.0
Loss after epoch 16: 132087.75
Loss after epoch 17: 131313.75
Loss after epoch 18: 129513.25
Loss after epoch 19: 128272.5
Loss after epoch 20: 127365.5
Loss after epoch 21: 126242.25
Loss after epoch 22: 125119.0
Loss after epoch 23: 124776.75
Loss after epoch 24: 124367.0
Loss after epoch 25: 123938.75
Loss after epoch 26: 122688.0
Loss after epoch 27: 112886.5
Loss after epoch 28: 109617.0
Loss after epoch 29: 109929.5
Loss after epoch 30: 108887.0
Loss after epoch 31: 108667.0
Loss after epoch 32: 

(1267241, 2270750)

In [20]:
w2v_model_cbow.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 408420.375
Loss after epoch 1: 322676.6875
Loss after epoch 2: 320022.1875
Loss after epoch 3: 320710.125
Loss after epoch 4: 315064.0
Loss after epoch 5: 304166.875
Loss after epoch 6: 273852.75
Loss after epoch 7: 258974.0
Loss after epoch 8: 256973.5
Loss after epoch 9: 254017.25
Loss after epoch 10: 252760.5
Loss after epoch 11: 249214.75
Loss after epoch 12: 249321.5
Loss after epoch 13: 246110.5
Loss after epoch 14: 236716.5
Loss after epoch 15: 215066.0
Loss after epoch 16: 214972.5
Loss after epoch 17: 215640.5
Loss after epoch 18: 214767.0
Loss after epoch 19: 214058.0
Loss after epoch 20: 214190.0
Loss after epoch 21: 213115.5
Loss after epoch 22: 212362.5
Loss after epoch 23: 212757.5
Loss after epoch 24: 212424.5
Loss after epoch 25: 211708.0
Loss after epoch 26: 212327.5
Loss after epoch 27: 211880.0
Loss after epoch 28: 210178.5
Loss after epoch 29: 210935.5
Loss after epoch 30: 210508.5
Loss after epoch 31: 211275.5
Loss after epoch 32: 209111.0
Loss 

(1982726, 3914200)

In [43]:
w2v_model_no_stopwords_cbow.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 263278.40625
Loss after epoch 1: 265707.71875
Loss after epoch 2: 261685.75
Loss after epoch 3: 262887.625
Loss after epoch 4: 256181.375
Loss after epoch 5: 255521.125
Loss after epoch 6: 254791.375
Loss after epoch 7: 254582.75
Loss after epoch 8: 244262.375
Loss after epoch 9: 242437.25
Loss after epoch 10: 242695.0
Loss after epoch 11: 242975.5
Loss after epoch 12: 242736.25
Loss after epoch 13: 242586.25
Loss after epoch 14: 243151.25
Loss after epoch 15: 242294.0
Loss after epoch 16: 238276.0
Loss after epoch 17: 226347.0
Loss after epoch 18: 225538.0
Loss after epoch 19: 224487.5
Loss after epoch 20: 225217.0
Loss after epoch 21: 225706.5
Loss after epoch 22: 225912.5
Loss after epoch 23: 224443.5
Loss after epoch 24: 225148.5
Loss after epoch 25: 224822.5
Loss after epoch 26: 224143.0
Loss after epoch 27: 223428.0
Loss after epoch 28: 223895.5
Loss after epoch 29: 223365.5
Loss after epoch 30: 222943.0
Loss after epoch 31: 222612.5
Loss after epoch 32: 22358

(2479003, 3914200)

### 4 - Ensayar

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["harry"], topn=12)

[('hagrid', 0.7141903638839722),
 ('ron', 0.6715407371520996),
 ('firenze', 0.6637693047523499),
 ('con', 0.6312284469604492),
 ('pero', 0.6172990202903748),
 ('y', 0.589058518409729),
 ('le', 0.5673465728759766),
 ('hermione', 0.563235878944397),
 ('lo', 0.5516765117645264),
 ('se', 0.5499004125595093),
 ('paraguas', 0.5367541313171387),
 ('su', 0.5364199280738831)]

In [23]:
# Palabras que MÁS se relacionan con...:
w2v_model_no_stopwords.wv.most_similar(positive=["harry"], topn=12)

[('ron', 0.7329361438751221),
 ('pie', 0.6225633025169373),
 ('hermione', 0.6189185380935669),
 ('firenze', 0.6048445701599121),
 ('salto', 0.5994933247566223),
 ('paraguas', 0.5888060927391052),
 ('hagrid', 0.570395827293396),
 ('razón', 0.5578779578208923),
 ('luego', 0.5564180612564087),
 ('serpiente', 0.5532912611961365),
 ('asiento', 0.5528450012207031),
 ('cuál', 0.5447615385055542)]

In [24]:
w2v_model_cbow.wv.most_similar(positive=["harry"], topn=12)

[('hagrid', 0.7141903638839722),
 ('ron', 0.6715407371520996),
 ('firenze', 0.6637693047523499),
 ('con', 0.6312284469604492),
 ('pero', 0.6172990202903748),
 ('y', 0.589058518409729),
 ('le', 0.5673465728759766),
 ('hermione', 0.563235878944397),
 ('lo', 0.5516765117645264),
 ('se', 0.5499004125595093),
 ('paraguas', 0.5367541313171387),
 ('su', 0.5364199280738831)]

In [44]:
w2v_model_no_stopwords_cbow.wv.most_similar(positive=["harry"], topn=12)

[('hermione', 0.803264856338501),
 ('ron', 0.7931991219520569),
 ('hagrid', 0.747541606426239),
 ('neville', 0.6497244834899902),
 ('pero', 0.6326873898506165),
 ('malfoy', 0.6138768196105957),
 ('dumbledore', 0.5999720692634583),
 ('dudley', 0.5966528654098511),
 ('decirle', 0.594443678855896),
 ('no', 0.5865117907524109),
 ('y', 0.5758406519889832),
 ('pared', 0.5426304340362549)]

In [26]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["harry"], topn=10)

[('doce', -0.025840453803539276),
 ('hay', -0.03424093499779701),
 ('llena', -0.043861888349056244),
 ('tendría', -0.05412810668349266),
 ('será', -0.05529558286070824),
 ('tengo', -0.05611425265669823),
 ('común', -0.06137216091156006),
 ('mente', -0.06494079530239105),
 ('además', -0.0684211328625679),
 ('podría', -0.06884966045618057)]

In [27]:
w2v_model_no_stopwords.wv.most_similar(negative=["harry"], topn=10)

[('puede', -0.08925294131040573),
 ('dentro', -0.11353717744350433),
 ('gringotts', -0.11999274790287018),
 ('realmente', -0.14371567964553833),
 ('era', -0.15000689029693604),
 ('dice', -0.1596834808588028),
 ('alumnos', -0.15982182323932648),
 ('familia', -0.17325185239315033),
 ('puertas', -0.17364320158958435),
 ('colegio', -0.1742624193429947)]

In [28]:
w2v_model_cbow.wv.most_similar(negative=["harry"], topn=10)

[('doce', -0.025840453803539276),
 ('hay', -0.03424093499779701),
 ('llena', -0.043861888349056244),
 ('tendría', -0.05412810668349266),
 ('será', -0.05529558286070824),
 ('tengo', -0.05611425265669823),
 ('común', -0.06137216091156006),
 ('mente', -0.06494079530239105),
 ('además', -0.0684211328625679),
 ('podría', -0.06884966045618057)]

In [45]:
w2v_model_no_stopwords_cbow.wv.most_similar(negative=["harry"], topn=10)

[('«', -0.021078286692500114),
 ('algún', -0.04624219238758087),
 ('quiso', -0.05066888779401779),
 ('puede', -0.06595911085605621),
 ('dice', -0.07718901336193085),
 ('parecía', -0.08614134043455124),
 ('habían', -0.08960120379924774),
 ('grande', -0.09509121626615524),
 ('lugar', -0.09595125913619995),
 ('caldero', -0.1069939136505127)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["mago"], topn=10)

[('secreto', 0.591467022895813),
 ('muggle', 0.5848466753959656),
 ('mamá', 0.5581204295158386),
 ('famoso', 0.5344323515892029),
 ('buscador', 0.5247362852096558),
 ('hogwarts', 0.524257242679596),
 ('nombre', 0.5236150622367859),
 ('eres', 0.5107726454734802),
 ('regalo', 0.496717631816864),
 ('es', 0.490953654050827)]

In [31]:
w2v_model_no_stopwords.wv.most_similar(positive=["mago"], topn=10)

[('muggle', 0.7022664546966553),
 ('eh', 0.6091876029968262),
 ('muerto', 0.599747359752655),
 ('sabe', 0.570197343826294),
 ('hace', 0.5546145439147949),
 ('quiso', 0.5446537137031555),
 ('tu', 0.5434936285018921),
 ('modos', 0.5424234867095947),
 ('famoso', 0.5420491099357605),
 ('mala', 0.5334197878837585)]

In [46]:
w2v_model_no_stopwords_cbow.wv.most_similar(positive=["mago"], topn=10)

[('muggle', 0.7223173975944519),
 ('secreto', 0.5571540594100952),
 ('tu', 0.5401334166526794),
 ('tampoco', 0.5370939373970032),
 ('dragón', 0.537052571773529),
 ('ésa', 0.5181748867034912),
 ('mamá', 0.5095603466033936),
 ('conseguido', 0.5077916383743286),
 ('buscador', 0.5040322542190552),
 ('falta', 0.5014690160751343)]

In [33]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["escoba"], topn=5)

[('vuelta', 0.6331589818000793),
 ('nota', 0.6122095584869385),
 ('nimbus', 0.6034864187240601),
 ('patada', 0.5982929468154907),
 ('última', 0.5704758763313293)]

In [34]:
w2v_model_no_stopwords.wv.most_similar(positive=["escoba"], topn=5)

[('saltó', 0.6066993474960327),
 ('harry', 0.5269842743873596),
 ('luego', 0.5193966031074524),
 ('libro', 0.47617170214653015),
 ('hermione', 0.4749956429004669)]

In [47]:
w2v_model_no_stopwords_cbow.wv.most_similar(positive=["escoba"], topn=5)

[('nimbus', 0.5997064709663391),
 ('hedwig', 0.556575357913971),
 ('2', 0.5508849024772644),
 ('vida', 0.5377461910247803),
 ('llave', 0.5369178652763367)]

In [48]:
w2v_model_no_stopwords_cbow.wv.most_similar(positive=["gryffindor"], topn=5)

[('hufflepuff', 0.6744503378868103),
 ('cincuenta', 0.6098018884658813),
 ('mañana', 0.5735531449317932),
 ('slytherin', 0.5522617101669312),
 ('ganado', 0.5411995649337769)]

### 5 - Visualizar agrupación de vectores

In [37]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [38]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [39]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model_no_stopwords)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [41]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model_no_stopwords_cbow)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Conclusiones

Se han obtenido vectores para las palabras del primer libro de Harry potter usando diferentes enfoques al momento de entrenar los modelos, excluyendo o no stopwords y con CBOW o Skipgram.

Tomando como referencia palabras clave que caracterizan el dataset y sus relacionadas, una primera impresión indicaría que, de los entrenados, el mejor modelo se obtuvo con una configuración CBOW excluyendo stopwords del dataset.
Palabras como "harry" se relacionan o tienen mayor similaridad con otros nombres de personajes del libro, "mago" denota la relación más fuerte con "muggle" que es su antónimo en la ficción, y "escoba" está asociada directamente con "nimbus", modelo de la escoba usada por Harry.

A su vez, en el espacio acotado de 2 dimensiones, se pueden apreciar clusters muy cerrados de palabras muy fuertemente asociadas entre ellas, como profesora y mcgonagall, tía y petunia, tío y vermont (que por contexto siempre figuran juntos), o slytheryn con gryffindor y puntos.